In [ ]:
import csv
import numpy as np
import tifffile as tf
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import ndimage
import pandas as pd
import os

In [ ]:
# Load the TIFF stack and extract shape information
file_path = 'paper_from_data.ome.tif'
stack = tf.imread(file_path)
shape = stack.shape
print('Loading Done')

In [ ]:
batch_size = 100  # Adjust the batch size as needed
total_indices = 2000
num_batches = total_indices // batch_size + (1 if total_indices % batch_size != 0 else 0)
origin = ((1000//2 - 1,  0, 0))
raw_time_series = stack[:2000, :, :]
print('Init')

for batch_idx in range(num_batches):
    start_index = batch_idx * batch_size
    end_index = min((batch_idx + 1) * batch_size, total_indices)

    processed_time_series_batch = np.zeros((end_index - start_index, shape[1], shape[2]))
    result_data_batch = np.zeros(((end_index // 2), shape[1], shape[2]))

    for index in range(start_index, end_index):
        # Your processing code for each index within the batch

        if index % 100 == 0:
            print('Filtering ' + str(index) + '/1500 done')

        # Perform filtering
        filtered = ndimage.uniform_filter(raw_time_series[:index + 1, :, :], size=(1000, 0, 0), mode='nearest',
                                           origin=origin)
        processed_time_series_batch[index - start_index, :, :] = filtered[index, :, :]
        
        if index % 2 == 0:
            blue_unf = raw_time_series[index, :, :]
            green_unf = raw_time_series[index - 1, :, :]
            blue_f = processed_time_series_batch[index - start_index, :, :]
            green_f = processed_time_series_batch[index - start_index - 1, :, :]

            # Avoid division by zero
            np.seterr(divide='ignore', invalid='ignore')
            blue_channel = np.divide(blue_unf, blue_f, out=np.zeros_like(blue_f), where=blue_f != 0)
            green_channel = np.divide(green_unf, green_f, out=np.zeros_like(green_f), where=green_f != 0)
            frame = np.divide(blue_channel, green_channel, out=np.zeros_like(green_channel), where=green_channel != 0)

            idx = int(index / 2)
            result_data_batch[idx, :, :] = frame